## Project Stage - III (Basic Machine Learning)

## Goals

The goal of Stage II is to utlize machine learning and statistical models to predict the trend of COVID-19 cases / deaths.

### Tasks:

#### Task 1: (70 pts)
- Team: (30)
    - Develop Linear and Non-Linear (polynomial) regression models for predicting cases and deaths in US. 
        - Start your data from the first day of infections in US. X-Axis - number of days since the first case, Y-Axis - number of new cases and deaths.
        - Calculate and report Root Mean Square Error (RMSE) for your models (linear and non-linear). Discuss bias versus variance tradeoff.
        - Plot trend line along for the data along with the forecast of 1 week ahead. 
        - Describe the trends as compared to other countries. 
- Member: (40 pts)
    - Utilize Linear and Non-Linear (polynomial) regression models to compare trends for a single state and its counties (top 5 with highest number of cases). Start your data from the first day of infections. 
        - X-Axis - number of days since the first case, Y - Axis number of new cases and deaths. Calcluate error using RMSE.
        - Identify which counties are most at risk. Model for top 5 counties with cases within a state and describe their trends.
        - Utilize the hospital data to calculate the point of no return for a state. Use percentage occupancy / utilization to see which states are close and what their trend looks like.
     - Perform hypothesis tests on questions identified in Stage II
        - e.x. *Does higher employment data (overall employment numbers) lead to higher covid case numbers or more rapid increase in covid cases.*. Here you would compare the covid cases to the state or county level enrichment data to prove or disprove your null hypothesis. In this case there will be a two tail - two sample t-test to see if there is a difference and then one-tail - two sample t-test to show higher or lower.
        - Depending on your type of data you can also perform Chi-square test for categorical hypothesis testing. 

    
#### Task 2: (30 pts)
- Member:
    - For each of the aforemention analysis plot graphs,
        - trend line
        - confidence intervals (error in prediction)
        - prediction path (forecast)

**Deliverable**
- Each member creates separate notebooks for member tasks. Upload all notebooks and reports to Github Repository. 
- Presentation recordings on canvas.

## Deadline: 11/18/2021

## Linear regression model

In [1]:
def country_calculation(df_all,country,date,day):
    df_country = df_all.copy()
    df_country = df_country.loc[df_country['Date'] >= date]
    df_country = df_country.loc[df_country['Country'] == country_dict[country]]
    features = ['iso_code', 'continent', 'location','total_cases', 'total_deaths', 'date']
    df_country = df_country[features]
    
    # Lags
    df_country = lag_feature(df_country, 'total_cases',range(1, 40))
    df_country = lag_feature(df_country, 'total_deaths', range(1,20))

    filter_col_confirmed = [col for col in df_country if col.startswith('total_cases')]
    filter_col_fatalities= [col for col in df_country if col.startswith('Fataliti')]
    filter_col = np.append(filter_col_confirmed, filter_col_fatalities)
    
    # Apply log transformation
    df_country[filter_col] = df_country[filter_col].apply(lambda x: np.log1p(x))
    df_country.replace([np.inf, -np.inf], 0, inplace=True)
    df_country.fillna(0, inplace=True) ####
    
    # Start/end of forecast
    start = df_country[df_country['Id']==-999].Day_num.min()
    end = df_country[df_country['Id']==-999].Day_num.max()
    #
    for d in range(start,end+1):
   
        X_train_1, X_train_2, Y_train_1, Y_train_2, x_test_1, x_test_2 = train_test_split_extend(df_country,d,day,filter_col_confirmed,filter_col_fatalities)
        
        regr_1, pred_1 = lin_reg(X_train_1, Y_train_1, x_test_1)
        df_country.loc[(df_country['Day_num'] == d) & (df_country['Country'] == country_dict[country]), 'total_cases'] = pred_1[0]
        
        regr_2, pred_2 = lin_reg(X_train_2, Y_train_2, x_test_2)
        df_country.loc[(df_country['Day_num'] == d) & (df_country['Country'] == country_dict[country]), 'Fatalities'] = pred_2[0]
        
        df_country = lag_feature(df_country, 'total_cases',range(1, 40))
        df_country = lag_feature(df_country, 'Fatalities', range(1,20))

        df_country.replace([np.inf, -np.inf], 0, inplace=True)
        df_country.fillna(0, inplace=True)
        
    print("Calculation done.")
    return df_country
    
    df_check = country_calculation(df_all, 'Germany', '2020-03-10', 48)